In [ ]:
import polars as pl

In [ ]:
!gsutil cp /home/eanegrin/buckets/b1/datasets/competencia_03.parquet /home/eanegrin/datasets/

In [ ]:
base_path = '/home/eanegrin/'
# base_path = 'C:/Eugenio/Maestria/DMEyF/'

dataset_path = base_path + 'datasets/'

dataset_file = 'competencia_03.parquet'

In [ ]:
df = pl.read_parquet(dataset_path + dataset_file)

In [ ]:
originales = df.select(pl.exclude(["numero_de_cliente", "foto_mes", "clase_ternaria"])).columns

In [ ]:
# corrijo el tipo de estas 2:

df = df.with_columns([
    pl.col("tmobile_app").cast(pl.Int64),
    pl.col("cmobile_app_trx").cast(pl.Int64)
])

In [ ]:
# lag 1
df = df.with_columns([
    pl.col(col)
    .sort_by(pl.col("foto_mes")) 
    .shift(1)
    .over("numero_de_cliente") 
    .alias(f"{col}_lag1")
    for col in originales
])

# lag 2
df = df.with_columns([
    pl.col(col)
    .sort_by(pl.col("foto_mes")) 
    .shift(2)
    .over("numero_de_cliente") 
    .alias(f"{col}_lag2")
    for col in originales
])

# delta 1
df = df.with_columns([
    (pl.col(col) - pl.col(f"{col}_lag1")).alias(f"{col}_delta1")
    for col in originales
])

# delta 2
df = df.with_columns([
    (pl.col(col) - pl.col(f"{col}_lag2")).alias(f"{col}_delta2")
    for col in originales
])